# Logistic Regression and Boosting Algorithms

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


## Predicting a Single Categorical Response
---



### Installing stuff

In [1]:
!pip install --upgrade textblob spacy 'gensim==4.2.0' swifter keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=ce62f16c790fb3276193078c4ffee56417b7fde9668db2fa3190592c472727b3
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
  Attempting uninstall: textblob
    Found existing installation: textblob 0.17.1
    Uninstalling textblob-0.17.1:
      Successfully uninstalled textblob-0.17.1
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2


In [2]:
!python -m textblob.download_corpora lite
!python -m spacy download en_core_web_sm

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Finished.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB         # Naive Bayes
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

import spacy
import gensim
import warnings
import nltk
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2024-05-09 10:10:07--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xds4lua69b7okw8/yelp.csv [following]
--2024-05-09 10:10:07--  https://www.dropbox.com/s/raw/xds4lua69b7okw8/yelp.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2b1726a7d447d02a3436b076ee.dl.dropboxusercontent.com/cd/0/inline/CSgLQHo-bmgMy4VL4p2vRCQy07pvkxGtZQn9-T_vQx4IgIO-5GBi4mOlk0c-NdhuEqValcvbY1xVAyeT9IbkePsTAML3iXlI4fZGF5oWoOfgX97o0uVvA7nOaCNuPaJIMof_6hW1tsA0q_72v8bBZjHl/file# [following]
--2024-05-09 10:10:08--  https://uc2b1726a7d447d02a3436b076ee.dl.dropboxusercontent.com/cd/0/inline/CSgLQHo-bmgMy4VL4p2vRCQy07pvkxGtZQn9-T_vQx4IgIO-5GBi4mOlk0c-NdhuEqValcvbY1xVAyeT9IbkePsTAML3iXlI4fZGF5oWoOfgX

In [15]:
# Read yelp.csv into a DataFrame.
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[ (yelp.stars == 1) | (yelp.stars == 5) ]

# Define X and y.
X = yelp_best_worst.text
y = yelp_best_worst.stars

# Split the new DataFrame into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

5307    5
5985    5
6918    1
4315    5
3356    5
       ..
2737    5
3142    5
2065    5
8596    1
7787    5
Name: stars, Length: 3268, dtype: int64
5307    If I could give it more than 5, I would.  Swee...
5985    We had a fantastic experience here! We went on...
6918    4 stars for the place itself and it's food/cof...
4315    I'm a huge fan of Padre's! What food I've trie...
3356    I love this place. I just tried the other Ethi...
                              ...                        
2737    Let me tell you about my first crush in Phoeni...
3142    My dear love and I went to the museum on a rom...
2065    You like hotdogs?  Motor (thats me) says get i...
8596    Nice facilities, nice AC, but two FATAL flaws:...
7787    The single best ribs I've ever had at any rest...
Name: text, Length: 3268, dtype: object


<a id="using-logistic-regression-for-classification"></a>
## Using Logistic Regression for Classification
---



In [18]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # You can adjust max_features as needed

# Fit and transform the text data to TF-IDF matrix
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Initialize logistic regression model
logreg = LogisticRegression()

# Fit the model
logreg.fit(X_train_tfidf, y_train)

LogisticRegression()

Of course this simply fails, we need to preprocess the text, convert it into a Tensor format and then and only then we can use models!

### Converting text to vectors

In [19]:
import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in textblob_tokenizer(text))
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    text_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords
    text_token_list = [word_rooter(word) if '#' not in word else word
                        for word in text_token_list] # apply word rooter
    text = ' '.join(text_token_list)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [48]:
# Apply the preprocessing to the dataset
import swifter
X_preprocessed = X.swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/4086 [00:00<?, ?it/s]

In [49]:
X_preprocessed[0]

'wife took birthday breakfast excel weather perfect made sit outsid overlook ground absolut pleasur waitress excel food arriv quickli semi busi saturday morn look like place fill pretti quickli earlier get better favor get bloodi mari phenomen simpli best ever pretti sure use ingredi garden blend fresh order amaz everyth menu look excel white truffl scrambl egg veget skillet tasti delici came piec griddl bread amaz absolut made meal complet best toast ever anyway ca wait go bac'

How do we pass from text to numbers? With tokenizers. We will use Tensorflow ones!

In [51]:
# Find a set named vocab that has all unique words
vocab = set()
for review in X_preprocessed:
  for word in textblob_tokenizer(review):
    if word not in vocab:
      vocab.add(word)

In [52]:
print(f'{len(vocab)} unique words')

13140 unique words


In [56]:
# Implement this method
def get_maximum_review_length(srs):
    maximum = 0  # Initialize the maximum review length
    # Iterate through each preprocessed text
    for reviews in srs:
      if len(textblob_tokenizer(review)) > maximum:
        maximum = len(textblob_tokenizer(review))
    return maximum

maximum = get_maximum_review_length(X_preprocessed)

In [57]:
print(f'The maximum review was {maximum} words long')

The maximum review was 45 words long


In [58]:
from tensorflow.keras.layers.experimental import preprocessing
ids_from_words = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [59]:
words_from_ids = preprocessing.StringLookup(
    vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

In [61]:
import tensorflow as tf
def text_from_ids(ids):
  return tf.strings.reduce_join(words_from_ids(ids), axis=-1, separator=' ')

In [66]:
ids = ids_from_words(preprocess_text('Only you can prevent forest fires', should_join=False))
ids

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([4073, 3534, 1416])>

In [63]:
preprocess_text('Only you can prevent forest fires', should_join=False)

['prevent', 'forest', 'fire']

In [64]:
text_from_ids(ids)


<tf.Tensor: shape=(), dtype=string, numpy=b'prevent forest fire'>

In [69]:
def pad_sequence_of_tokens(x, maxlen, unk_token='[UNK]'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

In [67]:
from keras_preprocessing.sequence import pad_sequences
# Very useful method to keep in mind
def get_ids_tensor(srs):

  processed = srs.swifter.apply(lambda x: pad_sequence_of_tokens(preprocess_text(x, should_join=False), maxlen=maximum)).to_list()
  return tf.squeeze(tf.constant(pad_sequences(ids_from_words(processed), maxlen=maximum, padding='post'), dtype='int32'))



In [70]:
all_ids = get_ids_tensor(srs=X_preprocessed.reset_index(drop=True))
all_ids

Pandas Apply:   0%|          | 0/4086 [00:00<?, ?it/s]

ValueError: Exception encountered when calling layer 'string_lookup_2' (type StringLookup).

Can't convert non-rectangular Python sequence to Tensor.

Call arguments received by layer 'string_lookup_2' (type StringLookup):
  • inputs=[["'wife'", "'took'", "'birthday'", "'breakfast'", "'excel'", "'weather'", "'perfect'", "'made'", "'sit'", "'outsid'", "'overlook'", "'ground'", "'absolut'", "'pleasur'", "'waitress'", "'excel'", "'food'", "'arriv'", "'quickli'", "'semi'", "'busi'", "'saturday'", "'morn'", "'look'", "'like'", "'place'", "'fill'", "'pretti'", "'quickli'", "'earlier'", "'get'", "'better'", "'favor'", "'get'", "'bloodi'", "'mari'", "'phenomen'", "'simpli'", "'best'", "'ever'", "'pretti'", "'sure'", "'use'", "'ingredi'", "'garden'", "'blend'", "'fresh'", "'order'", "'amaz'", "'everyth'", "'menu'", "'look'", "'excel'", "'white'", "'truffl'", "'scrambl'", "'egg'", "'veget'", "'skillet'", "'tasti'", "'delici'", "'came'", "'piec'", "'griddl'", "'bread'", "'amaz'", "'absolut'", "'m

In [45]:
all_ids.shape

TensorShape([4086, 477])

In [72]:
# Split the all_ids into train and test sets
X_train, X_test, y_train, y_test = train_test_split(all_ids.numpy(), y, test_size=0.2, random_state=42)


### Using Logistic Regression

In [73]:
from sklearn.metrics import classification_report
# Train a Logistic Regression on X_train and give the accuracy
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00       167
           5       0.80      1.00      0.89       651

    accuracy                           0.80       818
   macro avg       0.40      0.50      0.44       818
weighted avg       0.63      0.80      0.71       818



## Using Boosting Algorithms and other things

In [76]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

NameError: name 'classification_report' is not defined

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=50, learning_rate=0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

## Multiclass Classification

Just check in the estimators, most support multiclass classification.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, multi_class='multinomial').fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)

### **Homework**: Try to perform the stars classification with Logistic Regression but without filtering only for 5 and 1 stars.